## Overview

In this notebook, we'll go through the process of fine-tuning the LightGBM models in the `pdf-document-layout-analysis` service.

But before doing that, let's start with some basic concepts and introduce modules and methods to make the process easier and cleaner.

To begin with, you should first ensure that `Poppler` is installed on your system. We will use it to process PDFs:

In [1]:
%%bash

if ! command -v pdftohtml &> /dev/null
then
    echo "pdftohtml is not installed. Installing now..."
    sudo apt install pdftohtml
else
    echo "pdftohtml is already installed."
fi

pdftohtml is already installed.


We use Poppler to convert PDFs to XMLs. To work with Poppler in Python, we have created `PdfFeatures` module, which can be found in `pdf_features/PdfFeatures.py`.

In [2]:
from pdf_features.PdfFeatures import PdfFeatures

To open a PDF file with PdfFeatures module, simply write:

In [3]:
pdf_features: PdfFeatures = PdfFeatures.from_pdf_path("test_pdfs/regular.pdf")

Page-1
Page-2


When you open `pdf_features` like this, the XML file is saved in a temporary path and handled on the fly.

If you want to save the XML file, you should provide a path where it can be saved:

In [4]:
pdf_features: PdfFeatures = PdfFeatures.from_pdf_path("test_pdfs/regular.pdf", "test_pdfs/regular.xml")

Page-1
Page-2


Here is a part of the XML to illustrate what it looks like:

```
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE pdf2xml SYSTEM "pdf2xml.dtd">

<pdf2xml producer="poppler" version="23.04.0">
<page number="1" position="absolute" top="0" left="0" height="842" width="595">
	<fontspec id="0" size="10" family="JOIBEJ+Verdana" color="#000000"/>
	<fontspec id="1" size="10" family="JOIBGK+Verdana" color="#000000"/>
<text top="106" left="244" width="111" height="12" font="0"><b>RESOLUCIÓN DE LA </b></text>
<text top="118" left="157" width="284" height="12" font="0"><b>CORTE INTERAMERICANA DE DERECHOS HUMANOS </b></text>
<text top="129" left="227" width="145" height="12" font="0"><b>DEL 29 DE JULIO DE 1991 </b></text>
<text top="141" left="298" width="3" height="12" font="0"><b> </b></text>
<text top="153" left="298" width="3" height="12" font="0"><b> </b></text>
<text top="165" left="132" width="334" height="12" font="0"><b>MEDIDAS PROVISIONALES SOLICITADAS POR LA COMISIÓN </b></text>
<text top="177" left="177" width="245" height="12" font="0"><b>INTERAMERICANA DE DERECHOS HUMANOS </b></text>
<text top="188" left="225" width="149" height="12" font="0"><b>RESPECTO DE GUATEMALA </b></text>

...
```

When we convert PDFs to XMLs with Poppler, it creates `tokens`. These tokens are generally lines of text, but they can vary according to Poppler's heuristics and what has been extracted.  
A token can be a single character, empty string, or an entire line. Every `<text>` item you see above is a `token`.

The PdfFeatures module provides basic capabilities for working with PDF files. Here are some features of this module.  
You don't have to memorize them, but they can be useful for future reference:

- Every PdfFeatures instance has `pages` attribute. This attribute includes a list of `PdfPage` elements to work with each of the pages.
- Every PdfPage element has attributes like `page_number`, `page_width`, `page_height` and `tokens`.
- The `tokens` attribute includes a list of `PdfToken` elements to work with each of the tokens within that page.
- Every PdfToken element has attributes like `content`, `bounding_box`, `token_type`, `page_number`.
- The `content` attribute is, as the name implies, the string content of the given token.
- The`bounding_box` attribute stores the position of the given token on the page.
- `bounding_box` is a `Rectangle` element. For example, if you want to get the left coordinate of the token, you can do so by typing `token.bounding_box.left`. It will return an integer value.
- `token_type` attribute is for keeping the type of the token. It's a `TokenType` element and you'll see more details about this one in the next sections.
- Like PdfPage items, tokens also have a `page_number` attribute to indicate which page they are on. This is useful in some scenarios.

If you want to loop through the tokens of a file and check their contents you can use something like this:

In [5]:
for page in pdf_features.pages:
    for token in page.tokens:
        coordinates = [token.bounding_box.left, token.bounding_box.top, token.bounding_box.right, token.bounding_box.bottom]
        print(f"\033[96m{token.content}\033[0m \033[93m[Page: {page.page_number} || Coordinates: {coordinates}]\033[0m")

RESOLUCIÓN DE LA [Page: 1 || Coordinates: [244, 106, 355, 118]]
CORTE INTERAMERICANA DE DERECHOS HUMANOS [Page: 1 || Coordinates: [157, 118, 441, 130]]
DEL 29 DE JULIO DE 1991 [Page: 1 || Coordinates: [227, 129, 372, 141]]
MEDIDAS PROVISIONALES SOLICITADAS POR LA COMISIÓN [Page: 1 || Coordinates: [132, 165, 466, 177]]
INTERAMERICANA DE DERECHOS HUMANOS [Page: 1 || Coordinates: [177, 177, 422, 189]]
RESPECTO DE GUATEMALA [Page: 1 || Coordinates: [225, 188, 374, 200]]
CASO CHUNIMA [Page: 1 || Coordinates: [254, 224, 344, 236]]
LA CORTE INTERAMERICANA DE DERECHOS HUMANOS, [Page: 1 || Coordinates: [88, 259, 393, 271]]
VISTOS: [Page: 1 || Coordinates: [88, 295, 137, 307]]
1. [Page: 1 || Coordinates: [88, 318, 101, 330]]
La resolución del Presidente de la Corte Interamericana de Derechos Humanos [Page: 1 || Coordinates: [122, 318, 511, 330]]
de 15 de julio de 1991, sobre medidas provisionales solicitadas por la Comisión [Page: 1 || Coordinates: [88, 330, 514, 342]]
Interamericana de Derechos

## Fine-Tuning Models

Now that we have some overview about the `PdfFeatures` module, we can now start fine-tuning process.

In the `pdf-document-layout-analysis` service, there are two LightGBM (i.e. fast) models.

- The first model is used to determine the types of tokens. We call it `token_type_model`.
- The second model is used to identify the segments to which the tokens belong. We call this model `paragraph_extraction_model`.

The second model uses the predictions from the first model's output (predicted token types) as part of its features. So, let's start by fine-tuning the token type model.

### Fine-Tuning Token Type Model

#### Loading Data

To properly train a token type model, you should have a list of PdfFeatures items where the `token_type` attribute of their tokens is set correctly, as this attribute will be used as the label.

To see what labels are going to be used in the model, you can check `pdf_token_type_labels/TokenType.py`. As default, we are using the labels of [DocLayNet](https://github.com/DS4SD/DocLayNet) dataset.

In [6]:
def get_pdf_features_labels() -> PdfFeatures:
    # Assuming that you are loading your own labels in this part.
    # I'm just going to put a list with a single file for demonstration.
    pdf_features: PdfFeatures = PdfFeatures.from_pdf_path("test_pdfs/regular.pdf")
    labeled_pdf_features_list: list[PdfFeatures] = [pdf_features]
    return labeled_pdf_features_list

def train_token_type_model():
    model_configuration = ModelConfiguration()
    labeled_pdf_features_list: list[PdfFeatures] = get_pdf_features_labels()
    trainer = TokenTypeTrainer(labeled_pdf_features_list, model_configuration)
    train_labels = [token.token_type.get_index() for token in trainer.loop_tokens()]
    trainer.train("models/token_type_example_model.model", train_labels)    

train_token_type_model()

Don't forget to check what's inside the `model_configuration`. You might need to tune the hyperparameters.

If you want to use our trained models as base and refit with your own data, you can use this function:

In [ ]:
def refit_token_type_model():
    model_configuration = ModelConfiguration()
    model_configuration.resume_training = True
    labeled_pdf_features_list: list[PdfFeatures] = get_pdf_features_labels()
    trainer = TokenTypeTrainer(labeled_pdf_features_list, model_configuration)
    train_labels = [token.token_type.get_index() for token in trainer.loop_tokens()]
    trainer.train("models/token_type_lightgbm.model", train_labels)


Running this function will refit the same model with your data. Depending on your situation, it may or may not help you.

If it does not help, you can try to check other fine-tuning strategies in LightGBM. 

In that case, all you need to do is changing this part in `pdf_tokens_type_trainer/PdfTrainer.py` (lines 47-49):

```
        if self.model_configuration.resume_training and exists(model_path):
            model = lgb.Booster(model_file=model_path)
            gbm = model.refit(x_train, labels)
```

To make predictions with the trained model, you can use this function:

In [ ]:
def get_predictions():
    model_configuration = ModelConfiguration()
    pdf_features: PdfFeatures = PdfFeatures.from_pdf_path("test_pdfs/regular.pdf")
    trainer = TokenTypeTrainer([pdf_features], model_configuration)
    trainer.set_token_types()
    for token in pdf_features.pages[0].tokens[:20]:
        print(f"\033[96m{token.content}\033[0m \033[93m[{token.token_type}]\033[0m")

get_predictions()    

### Fine-Tuning Paragraph Extraction Model

#### Loading Data

The second model in our pipeline is the paragraph extraction model. After finding the type of each token, now, we are going to "group" the tokens, which means, we are going to find each token's segment.

We are going to explain the process but for this part, we highly recommend you to place your labeled data as in this following file structure and use the already existing methods. Otherwise, it can be a bit more harder for you to use our modules:

```
.
└── pdf-labeled-data
    ├── labeled_data
    │   ├── token_type
    │   │   ├── train_data
    │   │   │   ├── example_document1
    │   │   │   │   └── labels.json
    │   │   │   ├── example_document2
    │   │   │   │   └── labels.json
    │   │   │   └── example_document3
    │   │   │       └── labels.json
    │   │   └── test_data
    │   │       └── example_document4
    │   │           └── labels.json
    │   └── paragraph_extraction
    │       ├── train_data
    │       │   ├── example_document1
    │       │   │   └── labels.json
    │       │   ├── example_document2
    │       │   │   └── labels.json
    │       │   └── example_document3
    │       │       └── labels.json
    │       └── test_data
    │           └── example_document4
    │               └── labels.json
    └── pdfs
        ├── example_document1
        │   ├── document.pdf
        │   └── etree.xml
        ├── example_document2
        │   ├── document.pdf
        │   └── etree.xml
        ├── example_document3
        │   ├── document.pdf
        │   └── etree.xml
        └── example_document4
            ├── document.pdf
            └── etree.xml
```

Some details about this structure:

- Every detail in the token type labels file structure applies for this structure too.
- `paragraph_extraction` directory is where your paragraph extraction datasets are located, its name should not be something else.
- `token_type` labels are also shown in the structure because token types are used as a feature in the paragraph extraction model. If you do not have it, it will not break the pipeline and still train the model but the token_type feature for every token will be `TokenType.TEXT` in paragraph extractor model's features.
- If you do not have `token_type` labels, another option is, after loading the data, you can predict the token types with the token type model (will be shown below)


For labels.json files, they should have this structure:

```
{
    "pages": [
        {
            "number": 1,
            "labels": [
                {
                    "top": 86,
                    "left": 162,
                    "width": 292,
                    "height": 24,
                    "label_type": 0
                },
                {
                    "top": 122,
                    "left": 221,
                    "width": 174,
                    "height": 12,
                    "label_type": 0
                }
            ]
        },
        {
            "number": 2,
            "labels": [
                {
                    "top": 36,
                    "left": 296,
                    "width": 22,
                    "height": 13,
                    "label_type": 0
                },
                {
                    "top": 72,
                    "left": 71,
                    "width": 473,
                    "height": 49,
                    "label_type": 0
                }
            ]
        }
    ]
}
```

Here you see a list of labels for each page. Each label includes information about the coordinates `top`, `left`, `width`, `height` for each segment/paragraph. So, this time the coordinates belongs to the segments, not to the tokens.

As "label_type", it should be always 0 since there is only one type "paragraph" (don't get confused with this part, it's not important, just put 0 and go on).


Using this information, you can load your data like this:

In [ ]:
from os.path import join
from paragraph_extraction_trainer.PdfParagraphTokens import PdfParagraphTokens


def load_paragraph_extraction_labels():
	
	pdf_labeled_data_root_path = "path/to/pdf/labeled/data"
	datasets_path = join(pdf_labeled_data_root_path, "paragraph_extraction")
	labeled_data: list[PdfParagraphTokens] = []
	
	for dataset in listdir(join(datasets_path)):
		if "train" not in dataset:
			continue
		pdf_paragraph_tokens: PdfParagraphTokens = PdfParagraphTokens.from_labeled_data(pdf_labeled_data_root_path, dataset, pdf_name)
		labeled_data.append(pdf_paragraph_tokens)
	
	return labeled_data


from adapters.ml.pdf_tokens_type_trainer.TokenTypeTrainer import TokenTypeTrainer

def load_paragraph_extraction_labels():

    pdf_labeled_data_root_path = "path/to/pdf/labeled/data"
    datasets_path = join(pdf_labeled_data_root_path, "paragraph_extraction")
    labeled_pdf_paragraph_tokens_list: list[PdfParagraphTokens] = []
    
    for dataset in listdir(join(datasets_path)):
        if "train" not in dataset:
            continue
        pdf_paragraph_tokens: PdfParagraphTokens = PdfParagraphTokens.from_labeled_data(pdf_labeled_data_root_path, dataset, pdf_name)
        labeled_pdf_paragraph_tokens_list.append(pdf_paragraph_tokens)
    
    
        token_type_model_configuration = ModelConfiguration()
        labeled_pdf_features_list = [pdf_paragraph_tokens.pdf_features for pdf_paragraph_tokens in labeled_pdf_paragraph_tokens_list]
    trainer = TokenTypeTrainer(labeled_pdf_features_list, model_configuration)
    
    
    return labeled_pdf_paragraph_tokens_list

#### Fine-Tuning the Model

Again, to be able to use our trained paragraph extraction model, you should download it from our huggingface repo. You can just run `download_models.py` and both models are going to be downloaded.

If you want to download it manually, you can use this link: https://huggingface.co/HURIDOCS/pdf-document-layout-analysis/tree/main

After downloading it, place it into `models` directory. The path should be as follows:  
`~/pdf-document-layout-analysis/models/paragraph_extraction_lightgbm.model`

To train the paragraph extraction model from scratch:

In [ ]:
from paragraph_extraction_trainer.model_configuration import MODEL_CONFIGURATION


def loop_pdf_paragraph_tokens(pdf_paragraph_tokens_list: list[PdfParagraphTokens]):
    for pdf_paragraph_tokens in pdf_paragraph_tokens_list:
        for page in pdf_paragraph_tokens.pdf_features.pages:
            if not page.tokens:
                continue
            for token, next_token in zip(page.tokens, page.tokens[1:]):
                yield pdf_paragraph_tokens, token, next_token
            yield pdf_paragraph_tokens, page.tokens[-1], page.tokens[-1]


def train_paragraph_extraction_model():
    labeled_pdf_paragraph_tokens_list: list[PdfParagraphTokens] = load_paragraph_extraction_labels()
    labeled_pdf_features_list = [pdf_paragraph_tokens.pdf_features for pdf_paragraph_tokens in labeled_pdf_paragraph_tokens_list]
    trainer = ParagraphExtractorTrainer(labeled_pdf_features_list, MODEL_CONFIGURATION)
    
    train_labels = []
    for pdf_paragraph_tokens, token, next_token in loop_pdf_paragraph_tokens([pdf_paragraph_tokens]):
        train_labels.append(pdf_paragraph_tokens.check_same_paragraph(token, next_token))

    trainer.train("models/paragraph_extraction_example_model.model", train_labels)    

And to refit the model with your own data, all you need to do is setting `resume_training` configuration to `True`:

In [ ]:
def refit_paragraph_extraction_model():
    labeled_pdf_paragraph_tokens_list: list[PdfParagraphTokens] = load_paragraph_extraction_labels()
    labeled_pdf_features_list = [pdf_paragraph_tokens.pdf_features for pdf_paragraph_tokens in labeled_pdf_paragraph_tokens_list]
    MODEL_CONFIGURATION.resume_training = True
    trainer = ParagraphExtractorTrainer(labeled_pdf_features_list, MODEL_CONFIGURATION)
    
    train_labels = []
    for pdf_paragraph_tokens, token, next_token in loop_pdf_paragraph_tokens([pdf_paragraph_tokens]):
        train_labels.append(pdf_paragraph_tokens.check_same_paragraph(token, next_token))

    trainer.train("models/paragraph_extraction_example_model.model", train_labels)    

<font color='red'>[IMPORTANT]</font> If you want to use your own trained models in pdf-document-layout-analysis service, make sure their names are `token_type_lightgbm.model` and `paragraph_extraction_lightgbm.model` and are placed in `models` directory.

After finishing training, you can get the predictions of the model like shown in below:

In [ ]:
from pdf_tokens_type_trainer.ModelConfiguration import ModelConfiguration
from fast_trainer.model_configuration import MODEL_CONFIGURATION as PARAGRAPH_EXTRACTION_CONFIGURATION
from domain.PdfSegment import PdfSegment
from adapters.ml.fast_trainer.ParagraphExtractorTrainer import ParagraphExtractorTrainer

def get_predictions():
    pdf_features: PdfFeatures = PdfFeatures.from_pdf_path("test_pdfs/regular.pdf")
    # First, use token type model to find and set the types.
    token_type_trainer = TokenTypeTrainer([pdf_features], ModelConfiguration())
    token_type_trainer.set_token_types("models/token_type_lightgbm.model")
    trainer = ParagraphExtractorTrainer(pdfs_features=[pdf_features], model_configuration=PARAGRAPH_EXTRACTION_CONFIGURATION)
    segments: list[PdfSegment] = trainer.get_pdf_segments("models/paragraph_extraction_lightgbm.model")
    model_configuration = ModelConfiguration()
    for segment in segments[:20]:
        print(f"\033[96m{segment.text_content}\033[0m \033[93m[{segment.segment_type}]\033[0m \033[91m{segment.bounding_box.to_dict()}\033[0m")

Output of the `paragraph_extraction_model` is a list of `PdfSegment` items. Every item includes the information like `page_number`, `text_content`, `segment_type`, `bounding_box`, `pdf_name` for each of the segments. 

```
{
    "pages": [
        {
            "number": 1,
            "labels": [
                {
                    "top": 86,
                    "left": 162,
                    "width": 292,
                    "height": 24,
                    "label_type": 0
                },
                {
                    "top": 122,
                    "left": 221,
                    "width": 174,
                    "height": 12,
                    "label_type": 0
                }
            ]
        },
        {
            "number": 2,
            "labels": [
                {
                    "top": 36,
                    "left": 296,
                    "width": 22,
                    "height": 13,
                    "label_type": 0
                },
                {
                    "top": 72,
                    "left": 71,
                    "width": 473,
                    "height": 49,
                    "label_type": 0
                }
            ]
        }
    ]
}
```

Using this information, you can load your data like this:

In case you do not have `token_type` labels and want to find the types with the `token_type_model`, you can use this:

#### Fine-Tuning the Model

If you want to download it manually, you can use this link: https://huggingface.co/HURIDOCS/pdf-document-layout-analysis/tree/main

After downloading it, place it into `models` directory. The path should be as follows:  
`~/pdf-document-layout-analysis/models/paragraph_extraction_lightgbm.model`

In [12]:
from paragraph_extraction_trainer.model_configuration import MODEL_CONFIGURATION


def loop_pdf_paragraph_tokens(pdf_paragraph_tokens_list: list[PdfParagraphTokens]):
    for pdf_paragraph_tokens in pdf_paragraph_tokens_list:
        for page in pdf_paragraph_tokens.pdf_features.pages:
            if not page.tokens:
                continue
            for token, next_token in zip(page.tokens, page.tokens[1:]):
                yield pdf_paragraph_tokens, token, next_token
            yield pdf_paragraph_tokens, page.tokens[-1], page.tokens[-1]


def train_paragraph_extraction_model():
    labeled_pdf_paragraph_tokens_list: list[PdfParagraphTokens] = load_paragraph_extraction_labels()
    labeled_pdf_features_list = [pdf_paragraph_tokens.pdf_features for pdf_paragraph_tokens in labeled_pdf_paragraph_tokens_list]
    trainer = ParagraphExtractorTrainer(labeled_pdf_features_list, MODEL_CONFIGURATION)
    
    train_labels = []
    for pdf_paragraph_tokens, token, next_token in loop_pdf_paragraph_tokens([pdf_paragraph_tokens]):
        train_labels.append(pdf_paragraph_tokens.check_same_paragraph(token, next_token))

    trainer.train("models/paragraph_extraction_example_model.model", train_labels)    

In [13]:
def refit_paragraph_extraction_model():
    labeled_pdf_paragraph_tokens_list: list[PdfParagraphTokens] = load_paragraph_extraction_labels()
    labeled_pdf_features_list = [pdf_paragraph_tokens.pdf_features for pdf_paragraph_tokens in labeled_pdf_paragraph_tokens_list]
    MODEL_CONFIGURATION.resume_training = True
    trainer = ParagraphExtractorTrainer(labeled_pdf_features_list, MODEL_CONFIGURATION)
    
    train_labels = []
    for pdf_paragraph_tokens, token, next_token in loop_pdf_paragraph_tokens([pdf_paragraph_tokens]):
        train_labels.append(pdf_paragraph_tokens.check_same_paragraph(token, next_token))

    trainer.train("models/paragraph_extraction_example_model.model", train_labels)    

After finishing training, you can get the predictions of the model like shown in below:

Output of the `paragraph_extraction_model` is a list of `PdfSegment` items. Every item includes the information like `page_number`, `text_content`, `segment_type`, `bounding_box`, `pdf_name` for each of the segments. 